In [11]:
"""
具有“研究团队”主管智能体和“写作团队”主管智能体的分层系统
"""

import os
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langchain.agents import create_agent
from langchain_core.tools import tool

from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(
    model="qwen3-max-2026-01-23",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0,
)


@tool
def add(a: float, b: float) -> float:
    """两个数字相加"""
    return a + b


@tool
def multiply(a: float, b: float) -> float:
    """两个数字相乘"""
    return a * b


@tool
def web_search(query: str) -> str:
    """在网络上搜索信息"""
    return (
        "以下是 FAANG 公司 2024 年的员工人数：\n"
        "1. **Facebook (Meta)**: 67,317 名员工。"
        "2. **Apple**: 164,000 名员工。"
        "3. **Amazon**: 1,551,000 名员工。"
        "4. **Netflix**: 14,000 名员工。"
        "5. **Google (Alphabet)**: 181,269 名员工。"
    )


math_agent = create_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    system_prompt="你是一位数学专家。",
)

research_agent = create_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    system_prompt="你是一位世界一流的研究专家，可以进行网络搜索。不要做任何数学运算。",
)

research_team_supervisor = create_supervisor(
    agents=[research_agent, math_agent],
    model=model,
    prompt="你正在管理一个研究团队，该团队由研究专家和数学专家组成。研究专家能够利用网络搜索工具进行查询。",
)

research_team = research_team_supervisor.compile(name="research_team")


@tool
def write_report(topic: str) -> str:
    """撰写关于给定主题的报告"""
    return f"关于{topic}的报告：（报告的详细内容）"


@tool
def publish_report(report: str) -> str:
    """发布报告"""
    return f"报告已发布：{report}"


writing_agent = create_agent(
    model=model,
    tools=[write_report],
    name="writing_expert",
    system_prompt="你是一位写作专家。",
)

publish_agent = create_agent(
    model=model,
    tools=[publish_report],
    name="publish_expert",
    system_prompt="你是一位出版专家。",
)

writing_team_supervisor = create_supervisor(
    agents=[writing_agent, publish_agent],
    model=model,
    prompt="你正在管理一个写作团队，该团队由写作专家和出版专家组成。",
)

writing_team = writing_team_supervisor.compile(name="writing_team")

top_level_supervisor_agent = create_supervisor(
    agents=[research_team, writing_team],
    model=model,
    prompt="你是一位顶层主管，管理研究团队和写作团队。",
)

top_level_supervisor = top_level_supervisor_agent.compile(name="top_level_supervisor")

result = top_level_supervisor.invoke(
    {
        "messages": [
            {"role": "user", "content": "2024 年 FAANG 公司的总员工人数是多少？"}
        ]
    }
)

print(result["messages"][-1].content)

截至 2024 年，FAANG 公司的总员工人数为 **1,977,586 人**，具体分布如下：

- **Meta（原 Facebook）**：67,317 人  
- **Apple**：164,000 人  
- **Amazon**：1,551,000 人  
- **Netflix**：14,000 人  
- **Google（Alphabet）**：181,269 人  

如需进一步分析或数据来源，请随时告知！
